# Logging
Logging is essential to track events that happen during the execution of software.

Logs are used for two main purposes:
- To track standard processing tasks, or monitor user behavior (i.e. "*audit logging*") 
- To bringing attention to potential implementation issues or to assist in debugging (i.e. "*diagnostic logging*")



## Why not use print?
`print` can have it's uses for debugging very simple scripts but bigger, more complete software solutions will require proper logging solutions.

For one, log records created during a log event included additional **diagnostic information** such the filename, function name, time, etc.

Additionally, handlers can be created for different **levels of severity**. Making it easy to display low severity logs in the console, while persisting high severity logs in a file.

`print`, on the other hand, will always display to the console, and any additional context must be manually formatted and appended to the message.



## Log Levels
There are 5 standard log levels, in order of severity:

- DEBUG (10)
- INFO (20)
- WARNING (30)
- ERROR (40)
- CRITICAL (50)

Ok, there is actually a 6th level in Python called "NOTSET" (0) but it's basically just used to make inheritance from parent loggers more explicit.



### DEBUG
The "debug" log level is for **detailed output used for diagnostic processes**.

The output of such logs are usually suppresed in a production context. They might still be written to a log file to facilate debugging historical actions. Although more likely, they are temporarily enabled to resolve a specific issue or only displays during development.


### INFO
The "info" log level is used to **track normal operations**. They can be useful to confirm things are working as expected.

Similar to "debug" logs, the output of these logs might be suppresed in production in favor of being written to a log file.


### WARNING
"warning" logs are used to **indicate that something went wrong, something may go wrong or something important happened**. Conditions creating log warnings are not critical to the operation of the software but need to be brought to the developers' attention. Some examples would be: if a process is taking longer than expected, or flagging that disk space is running low.

These logs are more rarely suppressed.


### ERROR
Error logs are used to **flag serious problems in the operation of software**. 
They are used instead of raising an actual error in order to ensure the software keeps running.

These logs should never be suppressed.


### CRITICAL
Critical logs reflect a **very serious problem**. They should be used if the software is at risk of being unable to continue operating.

These logs should never be suppressed.

## When to log rather than raise an error?
Deciding whether an error should be suppresed by creating an error log vs raising the error is dependant on runtime context and the expected behavior of the software. 

For example, a manually executed script that encounters an error preventing further processing should raise an error in order to kill the process, and be informative enough for the developer to address the issue and relaunch the script.

On the other hand, if a script has to process hundreds of entries and it encounters an error at the Xth entry, it might be better to log that error, and finish processing the remaining entries. In such a situation, the information contained in the error should be sufficient for a developper to investigate (and hopefully fix) the issue, as well as isolate and re-process problematic entries. Some things to consider: entry identifier, what happened?, where in the code?, what action was taken? 


In the context of a long-runing process such as a webserver. Raising errors would kill the process and therefore extend the scope of impact to other users. In such a situation, the errors will almost always be suppressed and are very likely to be sent to a log monitoring solution.

Finally, particular consideration should be taken when the end-user is not a developper. Error messages should help them fix or report a problem. Cryptic references to source code are not useful in this context. Such details should be sent to a separate log, which can be cross-referenced with the user report to obtain a fuller picture during diagnosis.



## Example 1
2 functions are used to handle the calls to an api endpoint:


```python
def generate_filters(content):
    filters = get_default_filters()  # triggers a database request
    for c in content:
        filters.append(c.value)
    return filters

def api_request(query):
    return {
        "success": True,
        "filters": generate_filters(query.content)
    }
```


Improve this function by:

- Handling a `None` value in query.content
- Handling an error in `get_default_filters` when attempting to communicate with the database

Proposed solution:
```python
def generate_filters(content):
    filters = get_default_filters() # triggers a database request
    for c in content:
        try:
            filters.append(c.value)
        except Exception as e:
            logger.warn(f'There was a problem processing filters f{c}')
    return filters


def api_request(query):
    try:
        return {
            "success": True,
            "filters": generate_filters(query.content)
        }

    except Exception as e:
        logger.error(e, exc_info=True)
        return {
            "success": False,
            "filters": [],
            "message": "Failed to generate filters."
        }
```

## Example 2
A function loads a cached datastructure:
```python
def _get_pickled_data(name:str):
    f = open(f'cache/{name}.pickle', 'rb')
    users = pickle.load(f)
    f.close()
    return users
```

Improve this function by:
- Adding output to confirm the cache file was loaded
- Handle errors during loading


Proposed solution:
```python
def _get_pickled_data(name:str):
    try:
        f = open(f'cache/{name}.pickle', 'rb')
        users = pickle.load(f)
        f.close()
        logger.warn(f'Loaded "{name}" pickle')
        return users
    except FileNotFoundError:
        logger.warn(f'"{name}" pickle not found')
        return pd.DataFrame()
    except Exception as e:
        raise (e)
```


## Example 3

A function executes two processes and combines the result:

```python
def get_results(content, useExact, useSemantic):
    results = []
    if useExact:
        exact_results = search(content)
        results = exact_results

    if useSemantic:
        semantic_results = weaviate_search(content)
        results = results + semantic_results

    return results
```


Improve the function by:
- Helping a user monitor the progression of the function
- Helping a developper track the results of each operation


Proposed solution:
```python
def get_results(content, useExact, useSemantic):
    logger.info(f'Getting results for: "{query}" with exact={useExact} and semantic={useSemantic}...')

    results = []
    if useExact:
        exact_results = search(content)
        logger.debug(f"Number of exact results: {len(exact_results)}")
        results = exact_results

    if useSemantic:
        semantic_results = weaviate_search(content)
        logger.debug(f"Number of semantic results: {len(semantic_results)}")
        results = results + semantic_results

    logger.debug(f"Number of combined results {len(results)}")
    return results
```



## Standard streams
There are three interconnected, independant, input and output communication channels: **stdin**, **stdout**, **stderr**. 

stdin is not of interested in the context of logging since it relates to input. Whereas, stdout and stderr relate to output.

There isn't really a fundamental difference between stdout and stderr. The difference is one of convention.

This convention suggests that:
- **the final output** of an operation should be sent to **stdout**
- **stderr** should be used for **error messages or diagnostics information**

**Example**

You need to program a calculator script that handles basic additions.

Additionally, this calculator needs to monitor invokations for analytical perposes. This tracking is done by sending the invokation details to a webserver. Two messages are creating during this process: an initial "sending result to server...", and if an error is encountered, a "webserver not responding" messages.

**Question**

The script is asked to compute `1 + 2` but a network issue prevents communication with the webserver.

- Where should the result be sent? 
- Where should the analytics messages be sent?
- Are there any additional considerations?


Proposed solution:
- The answer (`3`) should be sent to stdout.
- Both messages ("sending result to server..." and "webserver not responding") should be sent to stderr as "INFO" and "ERROR" events, respectively.

By sending the messages to stderr, a user would be able to monitor progress and errors through stderr.

They would also still being able to process the output of the operation since stdout is not "polluted" by the logging messages. For example: by piping it to another process (`calc "2 + 1" | multiply "5"`).

### Redirecting streams
Each standard stream has a unique id which can be used to redirect stream output.

#### stdout (id = 1)
Redirect stdout by using `>` to overwrite, or `>>` to append.

Ex: `calc "2 + 1" > output.txt` would write "3" to output.txt.

#### stderr (id = 2)
Redirect stderr by using `2>` to overwrite, or `2>>` to append.

Ex: `calc "2 + 1" 2>> error.txt` would append "sending result to server...webserver not responding" to error.txt.

#### Both
To redirect stderr and stdout to different files: `calc "2 + 1" 1> output.txt 2> error.txt`

To redirect to the same file: `calc "2 + 1" &> output_and_error.txt`

#### Suppressing streams
The usual way to suppress output streams is to redirect them to `/dev/null`.

To suppress stderr output:
`calc "2 + 1" 2> /dev/null`

## Python logging
Python's logging module enables us to manage the processing and output for log events through the use of **handlers** and **formatters**.


To create a logger:

```python
import logging
logging.basicConfig()

```


To use it:
```python
logging.warning("Warning message")
logging.info("info message")

```

**NOTE** Because the default level is WARNING. Only log messages at or above the WARNING level will be printed. As a result, the "info message" will be suppressed. Calling `logging.basicConfig(level=logging.INFO)` would ensure info messages are processed.

#### Handlers
Log handlers **define the output destination** for log events. 

Handlers make it possible to display to the console, write to a file, send an email, send to a queue, and more. A list of handlers can be found in the Python docs: https://docs.python.org/3/howto/logging.html#useful-handlers.

Handlers have two key properties: a **formatter** and a **log level** which define how the log message should be formatted, and which level of logs should be handled.

By defining different handlers for different log levels, it is possible to, for example, write DEBUG logs to a file while displaying INFO, and WARNING logs, and writing ERROR and CRITICAL logs to a second file.

##### Write to a file with FileHandler
You can pass a `file` parameter when calling `basicConfig` to write log messages to a file.

The following example would write INFO and above logs to the "output.log" file:
```python
logging.basicConfig(level=logging.INFO, file="output.log")
```

Alternately, create it by calling `FileHandler`:
```python
logging.basicConfig(level=logging.INFO)
file_handler = logging.FileHandler("output.log")
```

You can use `TimedRotatingFileHandler` or `RotatingFileHandler` instead of `FileHandler` to automatically rotate log files based on a schedule or file size. For more on file handlers: https://docs.python.org/3/library/logging.handlers.html#logging.FileHandler

##### Redirect to console with StreamHandler
Use the `StreamHandler` to output to the console:
```
console_handler = logging.StreamHandler()
```

#### Formatters
Logs are only as useful as the information they contain. As such, it is important to include some context around the log messages. This can be accomplished with log formatting. A full list of available attributes is available here: https://docs.python.org/3/library/logging.html#logrecord-attributes

For most use cases, including the log level, message (with variable data) and the date/time is good enough:

```python
logging.basicConfig(format='%(asctime)s :: %(levelname)s :: %(message)s')
```

Adding more contextual data such as the filename generating the log (`%(filename)s`), the function generating the log (`%(funcName)s`), and the linenumber (`%(lineno)d`) may be relevant.

Alternatively, if traceback information is important for a particular log message, `logging.exception` should be used because it appends this information automatically or call `logging.error` with 	`exc_info` set to true (`logging.error(e, exc_info=True)`). 

If trying to obtain the stack trace for an unhandled exception, import the `traceback` module and call ` logging.error("uncaught exception: %s", traceback.format_exc())`.

### Using a custom logger
So far logging was being done through the default (also known as "root") logger. This is generally accepted to be an anti-pattern since it means all modules use the same logger, and becomes particularly problematic since `logging.basicConfig` cannot be changed once set. Imported modules would therefore be unable to modify the logger, and all logs would end up in the same outputs regardless of their origin.

#### Create a custom logger
Calling `logging.getLogger` will either create a new logger or return an existing one if a logger with the given name already exists. 

```python
logger = logging.getLogger(__name__)
logger.info('module log message')
```

Using the `__name__` variable as a parameter means the logger will be named after the module, and thus facilitates debugging.

It is important to note that loggers have a hierarchy, and the root logger is at the top of this hierarchy. Any properties set on a parent logger will be applied to the child modules unless overriden. As such, if the root logger has `level=logging.ERROR`, and your custom logger does not set a level. The child logger will inherit `level=logging.ERROR` from the root logger, rather than the usual default of WARNING.

Additionally, the logger hierarchy follows the dot notation. If a "bob" logger is created, and then a "bob.burger" logger is created. The "bob.burger" will be a child of the "bob" logger, and inherit it's properties. An exception being that if "bob.burger" is created before "bob", both will have "root" as their parent.


### Configuring with a file
Logging comes with a `dictConfig` method, which makes it possible to define loggers using a dictionary. There is also a `fileConfig` method but the official Python recomendation is to use `dictConfig`.


A simple configuration file looks like:
```python
LOG_CONFIG = {
    'version': 1,
    'disable_existing_loggers': False,
    'formatters': {
        'default_formatter': {
            'format': '[%(levelname)s:%(asctime)s] %(message)s'
        },
    },
    'handlers': {
        'stream_handler': {
            'class': 'logging.StreamHandler',
            'formatter': 'default_formatter',
        },
    },
    'loggers': {
        'custom_logger': {
            'handlers': ['stream_handler'],
            'level': 'INFO',
            'propagate': False
        }
    }
}
```

It is rather important to include the `disable_existing_loggers` parameter since the default value is `True` and will result in any non-root loggers created before the call to `dictConfig` to be disabled.

It is advised to define handler behavior as high in the logger hierarchy as possible, and let the configurations propagate to lower lever loggers rather than redefining them individually.

Setting `propagate` to False prevents the events logged to this logger to be passed to the handlers of higher level (ancestor) loggers.
Otherwise, messages are passed directly to the ancestor loggers’ handlers - neither the level nor filters of the ancestor loggers in question are considered.

To use the previously defined configuration file:
```python
import logging.config
from .config import LOG_CONFIG

logging.config.dictConfig(LOG_CONFIG)
logger = logging.getLogger(__name__)
```

### Examples
#### 1- Output all logs to console, write ERROR and above to file
http://localhost:8888/notebooks/Example%201.ipynb


#### 2- Output ERROR logs for a module to a different file
http://localhost:8888/notebooks/Example%202.ipynb


#### 3- Override imported logger configuration
http://localhost:8888/notebooks/Example%203.ipynb

### Python best practices

#### Always use custom loggers
Creating custom loggers by using `getLogger(__name__)` automatically creates a hierarchy throughout your codebase, and makes the debugging process much cleaner.

For example if your code has the following structure:
```python
src/
	- main.py
	- module/
		- process.py
		- submodule/
			- subprocess.py

```
Create a single definition for a `module` logger.
Loggers created in process.py (__name__ = `module.process`) and subprocess.py (__name__ = `module.submodule.subprocess`) would inherit all of it's properties.


#### Always configure root logger
While you should never use the root logger, you should always configure it to make it easier to track errors generated by libraries which have not declared custom loggers.

By default the root logger only logs to stderr.

#### Think about debugging and end-user context
The needs of the end user and developper are not always aligned. Make sure your log events contain the necessary information for users to act on an error, as well as for developpers to investigate and resolve them.